In [1]:
import pandas as pd
import os

In [2]:
def preprocess(file_path):
    import re
    import os
    match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
    date = pd.to_datetime(match.group(0))
    
    sheet = pd.read_excel(file_path, header=None)
    locations = sheet.iloc[3, 5:35].dropna().values
    environment_headers = ['time', '수평면', '외기온도', '경사면', '모듈온도']

    location_headers = []
    for location in locations:
        location_headers.append(location)

    final_headers = environment_headers + location_headers
    
    final_data_corrected = sheet.iloc[6:30, [0, 1, 2, 3, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34]]
    
    final_data_corrected.columns = final_headers
    final_data_corrected.insert(0, 'day', date)
    final_data_corrected['time'] = final_data_corrected['time'].str.replace('시', '').str.strip().astype(int)
    
    new_file_path = os.path.join(os.path.dirname(file_path), 'processed', os.path.basename(file_path)[:-4] + '_filtered.csv')
    final_data_corrected.to_csv(new_file_path, index=False)
    
    return new_file_path


In [3]:
file_dir = './solar-power-report/'

file_list = [f for f in os.listdir(file_dir) if os.path.isfile(os.path.join(file_dir, f))]
file_list

['태양광 일보.gcf_2022-08-02_23-59-22.xls',
 '태양광 일보.gcf_2022-07-24_23-59-04.xls',
 '태양광 일보.gcf_2022-07-30_23-59-16.xls',
 '태양광 일보.gcf_2022-08-21_23-59-09.xls',
 '태양광 일보.gcf_2022-08-30_23-59-28.xls',
 '태양광 일보.gcf_2022-07-06_23-59-28.xls',
 '태양광 일보.gcf_2022-07-01_23-59-28.xls',
 '태양광 일보.gcf_2022-08-14_23-59-16.xls',
 '태양광 일보.gcf_2022-08-20_23-59-27.xls',
 '태양광 일보.gcf_2022-07-25_23-59-16.xls',
 '태양광 일보.gcf_2022-08-25_23-59-28.xls',
 '태양광 일보.gcf_2022-08-31_23-59-10.xls',
 '태양광 일보.gcf_2022-07-07_23-59-10.xls',
 '태양광 일보.gcf_2022-07-12_23-59-10.xls',
 '태양광 일보.gcf_2022-08-17_23-59-21.xls',
 '태양광 일보.gcf_2022-08-06_23-59-10.xls',
 '태양광 일보.gcf_2022-08-01_23-59-10.xls',
 '.DS_Store',
 '태양광 일보.gcf_2022-07-22_23-59-10.xls',
 '태양광 일보.gcf_2022-08-13_23-59-04.xls',
 '태양광 일보.gcf_2022-07-03_23-59-22.xls',
 '태양광 일보.gcf_2022-08-16_23-59-09.xls',

In [4]:
merged_df = pd.read_csv('./merged_dataset.csv')
merged_df['day'] = pd.to_datetime(merged_df['day'])

temp_df_path = preprocess(os.path.join(file_dir, file_list[0]))
temp_df = pd.read_csv(temp_df_path)
temp_df['day'] = pd.to_datetime(temp_df['day'])
merged_df = pd.merge(merged_df, temp_df, on=['day', 'time'], how='left')

for file_name in file_list[1:]:
  if file_name.startswith('.'):
    continue
  temp_df = pd.read_csv(preprocess(os.path.join(file_dir, file_name)))
  for index, row in temp_df.iterrows():
      mask = (merged_df['day'] == row['day']) & (merged_df['time'] == row['time'])
      for column in temp_df.columns[2:]:
          # Updating only the columns that are present in both DataFrames
          if column in merged_df.columns:
              merged_df.loc[mask, column] = row[column]
  
merged_df.to_csv('merged_result.csv', index=False)